In [3]:
# import section

import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [4]:
# Data loading function

def LoadData():
    year = 1972
    total_data = {}
    years = []
    for i in range(28):
        if year == 1979 or year == 1981 or year == 1984 or year == 1986 or year ==1992:
            year += 1
        years.append(str(year))
        print year,
        data = pd.io.stata.read_stata("/Users/jorrenbosga/Desktop/AUC/Masterclass/MasterclassBDProject/GSS" + str(year) + ".DTA")
        total_data[str(year)] = data
        year += 1
        if year > 1994:
            year += 1
        
    print "Data Loaded"
    return total_data

total_data = LoadData()

1972 1973 1974 1975 1976 1977 1978 1980 1982 1983 1985 1987 1988 1989 1990 1991 1993 1994 1996 1998 2000 2002 2004 2006 2008 2010 2012 2014 Data Loaded


In [89]:
def AllAnswers(category):
    for year in total_data.keys():
        if category not in total_data[year].keys():
            continue
        else:
            cat = getattr(total_data[year], category)

            cont_subcats = []
            for subcat in cat:
                if subcat not in cont_subcats:
                    cont_subcats.append(subcat)
                    
    return cont_subcats



In [93]:
# Counting function
# Returns dictionary with frequency of certain answer in a chosen category

def Count(year, category):
    
    filtered = getattr(total_data[year], category)
    counted = {}
    for answer in filtered:
        if answer not in counted:
            counted[answer] = 1
        else:
            counted[answer] = counted[answer] + 1   
    return counted

def CategoryFilter(year, category):
    incomplete = Count(year, category)
    totalcats = AllAnswers(category)
    for category in totalcats:
        if category not in incomplete.keys():
            incomplete[category] = 1e-16
    return incomplete
    
# Proportion function
# Returns list with proportions based on counting function
def Proportions(year, category):  # input should be output of Count function
    counted = CategoryFilter(year, category)
    numbervals = counted.values()
    totalval = 0
    proportions = []
    for item in numbervals:
        totalval += item
    for item in numbervals:
        proportions.append(float(item) / float(totalval))
        
    return proportions



{nan: 5, 'catholic': 327, 'none': 109, 'christian': 1e-16, 'jewish': 27, 'protestant': 862, 'INTER-NONDENOMINATIONAL': 1e-16, 'buddhism': 1e-16, 'other': 42, 'MOSLEM/ISLAM': 1e-16, 'OTHER EASTERN': 1e-16, 'hinduism': 1e-16, 'NATIVE AMERICAN': 1e-16, 'ORTHODOX-CHRISTIAN': 1e-16}


In [95]:
def CrossCount(year, category1, category2, subcategory):
    filtered = getattr(total_data[year], category1)[(getattr(total_data[year], category2)) == subcategory]
    counted = {}
    for answer in filtered:
        if answer not in counted:
            counted[answer] = 1
        else:
            counted[answer] = counted[answer] + 1   
    return counted

def CrossCategoryFilter(year, category1, category2, subcategory):
    incomplete = CrossCount(year, category1, category2, subcategory)
    totalcats = AllAnswers(category1)
    for category in totalcats:
        if category not in incomplete.keys():
            incomplete[category] = 1e-16
    return incomplete

def CrossProportions(year, category1, category2, subcategory):
    counted = CrossCategoryFilter(year, category1, category2, subcategory)
    numbervals = counted.values()
    totalval = 0
    proportions = []
    for item in numbervals:
        totalval += item
    for item in numbervals:
        proportions.append(float(item) / float(totalval))
        
    return proportions

# Entropy function

def Entropy(year, category1, category2, subcategory):
    param1 = CrossProportions(year, category1, category2, subcategory)
    param2 = Proportions(year, category1)
    if len(param1) != len(param2):
        params = [param1, param2]
        lengths = [len(param1), len(param2)]
        maxlength = max(lengths)    
        for param in params:
            if len(param) < maxlength:
                for i in range(maxlength - len(param)):
                    param.append(1*10**-20)
    entropy = sp.stats.entropy(param1, param2)
    return entropy



{nan: 543, 'SOMETIMES WRONG': 185, 'ALWAYS WRONG': 377, 'NOT WRONG AT ALL': 316, 'ALMST ALWAYS WRG': 100}


2      SOMETIMES WRONG
4     NOT WRONG AT ALL
5                  NaN
6     NOT WRONG AT ALL
14    NOT WRONG AT ALL
18    ALMST ALWAYS WRG
19                 NaN
21    NOT WRONG AT ALL
27                 NaN
28                 NaN
29                 NaN
30    ALMST ALWAYS WRG
32                 NaN
33                 NaN
40    NOT WRONG AT ALL
...
1350        ALWAYS WRONG
1351    NOT WRONG AT ALL
1352                 NaN
1353                 NaN
1354                 NaN
1355                 NaN
1356                 NaN
1357        ALWAYS WRONG
1364                 NaN
1365        ALWAYS WRONG
1366                 NaN
1367    NOT WRONG AT ALL
1368        ALWAYS WRONG
1369    ALMST ALWAYS WRG
1370    NOT WRONG AT ALL
Name: premarsx, Length: 862, dtype: object


0      SOMETIMES WRONG
1                  NaN
2     ALMST ALWAYS WRG
3     NOT WRONG AT ALL
4     NOT WRONG AT ALL
5                  NaN
6     NOT WRONG AT ALL
7                  NaN
8     NOT WRONG AT ALL
9         ALWAYS WRONG
10     SOMETIMES WRONG
11                 NaN
12    ALMST ALWAYS WRG
13                 NaN
14    NOT WRONG AT ALL
...
2802                 NaN
2803        ALWAYS WRONG
2804    ALMST ALWAYS WRG
2805                 NaN
2806    NOT WRONG AT ALL
2807    NOT WRONG AT ALL
2808        ALWAYS WRONG
2809     SOMETIMES WRONG
2810                 NaN
2811     SOMETIMES WRONG
2812                 NaN
2813    NOT WRONG AT ALL
2814    ALMST ALWAYS WRG
2815                 NaN
2816        ALWAYS WRONG
Name: premarsx, Length: 2817, dtype: object
